In [3]:
!pip3 install -U ipytest

Defaulting to user installation because normal site-packages is not writeable


In [4]:
import pytest
import ipytest

ipytest.autoconfig()
ipytest.config.rewrite_asserts = True

__file__ = 'test_fclayer.ipynb'

import numpy as np
from onnx import TensorProto, helper
from finn.util.visualization import showSrc, showInNetron

from finn.custom_op.registry import getCustomOp
import finn.core.onnx_exec as oxe
import finn.custom_op.general.xnorpopcount as xp
from finn.analysis.fpgadataflow.hls_synth_res_estimation import hls_synth_res_estimation
from finn.core.datatype import DataType
from finn.core.modelwrapper import ModelWrapper
from finn.custom_op.general.multithreshold import multithreshold
from finn.transformation.fpgadataflow.prepare_ip import PrepareIP
from finn.transformation.fpgadataflow.prepare_cppsim import PrepareCppSim
from finn.transformation.fpgadataflow.compile_cppsim import CompileCppSim
from finn.transformation.fpgadataflow.hlssynth_ip import HLSSynthIP
from finn.transformation.fpgadataflow.set_exec_mode import SetExecMode
from finn.transformation.general import GiveReadableTensorNames, GiveUniqueNodeNames
from finn.transformation.fpgadataflow.prepare_rtlsim import PrepareRTLSim
from finn.util.basic import calculate_signed_dot_prod_range, gen_finn_dt_tensor
from finn.analysis.fpgadataflow.exp_cycles_per_layer import exp_cycles_per_layer
from finn.analysis.fpgadataflow.dataflow_performance import dataflow_performance
from finn.analysis.fpgadataflow.res_estimation import res_estimation

from finn.transformation.fpgadataflow.insert_fifo import InsertFIFO
from finn.util.basic import pynq_part_map
from finn.transformation.fpgadataflow.make_zynq_proj import ZynqBuild
from finn.analysis.fpgadataflow.post_synth_res import post_synth_res

build_dir = "/workspace/finn" 

In [5]:
def make_single_fclayer_modelwrapper(W, pe, simd, wdt, idt, odt, T=None, tdt=None):
    mw = W.shape[0]
    mh = W.shape[1]
    assert mh % pe == 0
    assert mw % simd == 0

    # there are two ways to implement bipolar weights and inputs for
    # StreamingFC:
    # - specify their datatypes as such
    # - specify their datatypes as BINARY as use binaryXnorMode
    if wdt == DataType.BIPOLAR and idt == DataType.BIPOLAR:
        # we'll internally convert weights/inputs to binary and specify the
        # datatypes as such, and also set the binaryXnorMode attribute to 1
        export_wdt = DataType.BINARY
        export_idt = DataType.BINARY
        binary_xnor_mode = 1
    else:
        export_wdt = wdt
        export_idt = idt
        binary_xnor_mode = 0

    inp = helper.make_tensor_value_info("inp", TensorProto.FLOAT, [1, mw])
    outp = helper.make_tensor_value_info("outp", TensorProto.FLOAT, [1, mh])
    if T is not None:
        no_act = 0
        node_inp_list = ["inp", "weights", "thresh"]
        if odt == DataType.BIPOLAR:
            actval = 0
        else:
            actval = odt.min()
    else:
        # no thresholds
        node_inp_list = ["inp", "weights"]
        actval = 0
        no_act = 1
    FCLayer_node = helper.make_node(
        "StreamingFCLayer_Batch",
        node_inp_list,
        ["outp"],
        domain="finn.custom_op.fpgadataflow",
        backend="fpgadataflow",
        MW=mw,
        MH=mh,
        SIMD=simd,
        PE=pe,
        inputDataType=export_idt.name,
        weightDataType=export_wdt.name,
        outputDataType=odt.name,
        ActVal=actval,
        binaryXnorMode=binary_xnor_mode,
        noActivation=no_act,
    )
    graph = helper.make_graph(
        nodes=[FCLayer_node], name="fclayer_graph", inputs=[inp], outputs=[outp]
    )

    model = helper.make_model(graph, producer_name="fclayer-model")
    model = ModelWrapper(model)

    model.set_tensor_datatype("inp", idt)
    model.set_tensor_datatype("outp", odt)
    model.set_tensor_datatype("weights", wdt)
    if binary_xnor_mode:
        # convert bipolar to binary
        model.set_initializer("weights", (W + 1) / 2)
    else:
        model.set_initializer("weights", W)
    if T is not None:
        model.set_tensor_datatype("thresh", tdt)
        model.set_initializer("thresh", T)
    return model

In [6]:
def prepare_inputs(input_tensor, idt, wdt):
    if wdt == DataType.BIPOLAR and idt == DataType.BIPOLAR:
        # convert bipolar to binary
        return {"inp": (input_tensor + 1) / 2}
    else:
        return {"inp": input_tensor}

In [ ]:
# mem_mode: const or decoupled
@pytest.mark.parametrize("mem_mode", ["const"])
# activation: None or DataType
@pytest.mark.parametrize("act", [DataType.BIPOLAR, DataType.INT2, DataType.INT4])
# weight datatype
@pytest.mark.parametrize("wdt", [DataType.BIPOLAR, DataType.INT2, DataType.INT4])
# input datatype
@pytest.mark.parametrize("idt", [DataType.BIPOLAR, DataType.INT2, DataType.INT4])
# neuron folding, -1 is maximum possible
@pytest.mark.parametrize("nf", [-1, 2, 1])
# synapse folding, -1 is maximum possible
@pytest.mark.parametrize("sf", [-1, 2, 1])
# HLS matrix width (input features)
@pytest.mark.parametrize("mw", [8])
# HLS matrix height (output features)
@pytest.mark.parametrize("mh", [8])
@pytest.mark.slow
@pytest.mark.vivado
def test_fpgadataflow_fclayer_cppsim(mem_mode, idt, wdt, act, nf, sf, mw, mh):
    if nf == -1:
        nf = mh
    if sf == -1:
        sf = mw
    pe = mh // nf
    simd = mw // sf
    assert mh % pe == 0
    assert mw % sf == 0
    # generate weights
    W = gen_finn_dt_tensor(wdt, (mw, mh))
    # generate input data
    x = gen_finn_dt_tensor(idt, (1, mw))
    if act is None:
        # no activation, produce accumulators
        T = None
        tdt = None
        if wdt == DataType.BIPOLAR and idt == DataType.BIPOLAR:
            odt = DataType.UINT32
        else:
            odt = DataType.INT32
    else:
        odt = act
        (min, max) = calculate_signed_dot_prod_range(idt, wdt, mw)
        n_steps = act.get_num_possible_values() - 1
        T = np.random.randint(min, max - 1, (mh, n_steps)).astype(np.float32)
        # provide non-decreasing thresholds
        T = np.sort(T, axis=1)
        # generate thresholds for activation
        if wdt == DataType.BIPOLAR and idt == DataType.BIPOLAR:
            tdt = DataType.UINT32
            # bias thresholds to be positive
            T = np.ceil((T + mw) / 2)
            assert (T >= 0).all()
        else:
            tdt = DataType.INT32
    model = make_single_fclayer_modelwrapper(W, pe, simd, wdt, idt, odt, T, tdt)
    for node in model.graph.node:
        # lookup op_type in registry of CustomOps
        inst = getCustomOp(node)
        inst.set_nodeattr("mem_mode", mem_mode)
    model = model.transform(SetExecMode("cppsim"))
    model = model.transform(PrepareCppSim())
    model = model.transform(CompileCppSim())
    # prepare input data
    input_dict = prepare_inputs(x, idt, wdt)
    if wdt == DataType.BIPOLAR and idt == DataType.BIPOLAR:
        # convert inputs to binary and use xnorpopcountmatmul
        y = xp.xnorpopcountmatmul((x + 1) / 2, (W + 1) / 2)
    else:
        y = np.matmul(x, W)
    if T is not None:
        y = multithreshold(y, T)
        if act == DataType.BIPOLAR:
            # binary to bipolar
            y = 2 * y - 1
        else:
            # signed offset
            y += act.min()
    oshape = model.get_tensor_shape("outp")
    y_expected = y.reshape(oshape)
    # execute model
    y_produced = oxe.execute_onnx(model, input_dict)["outp"]

    y_produced = y_produced.reshape(y_expected.shape)

    assert (y_produced == y_expected).all(), "cppsim failed"

In [ ]:
%%ipytest

# mem_mode: const or decoupled
@pytest.mark.parametrize("mem_mode", ["const"])
# activation: None or DataType
@pytest.mark.parametrize("act", [DataType.BIPOLAR, DataType.INT2, DataType.INT4])
# weight datatype
@pytest.mark.parametrize("wdt", [DataType.BIPOLAR, DataType.INT2, DataType.INT4])
# input datatype
@pytest.mark.parametrize("idt", [DataType.BIPOLAR, DataType.INT2, DataType.INT4])
# neuron folding, -1 is maximum possible
@pytest.mark.parametrize("nf", [-1, 2, 1])
# synapse folding, -1 is maximum possible
@pytest.mark.parametrize("sf", [-1, 2, 1])
# HLS matrix width (input features)
@pytest.mark.parametrize("mw", [8])
# HLS matrix height (output features)
@pytest.mark.parametrize("mh", [8])
@pytest.mark.slow
@pytest.mark.vivado

def test_fpgadataflow_fclayer_rtlsim(mem_mode, idt, wdt, act, nf, sf, mw, mh):
    if nf == -1:
        nf = mh
    if sf == -1:
        sf = mw
    pe = mh // nf
    simd = mw // sf
    assert mh % pe == 0
    assert mw % sf == 0
    # generate weights
    W = gen_finn_dt_tensor(wdt, (mw, mh))
    # generate input data
    x = gen_finn_dt_tensor(idt, (1, mw))
    if act is None:
        # no activation, produce accumulators
        T = None
        tdt = None
        if wdt == DataType.BIPOLAR and idt == DataType.BIPOLAR:
            odt = DataType.UINT32
        else:
            odt = DataType.INT32
    else:
        odt = act
        (min, max) = calculate_signed_dot_prod_range(idt, wdt, mw)
        n_steps = act.get_num_possible_values() - 1
        T = np.random.randint(min, max - 1, (mh, n_steps)).astype(np.float32)
        # provide non-decreasing thresholds
        T = np.sort(T, axis=1)
        # generate thresholds for activation
        if wdt == DataType.BIPOLAR and idt == DataType.BIPOLAR:
            tdt = DataType.UINT32
            # bias thresholds to be positive
            T = np.ceil((T + mw) / 2)
            assert (T >= 0).all()
        else:
            tdt = DataType.INT32
    model = make_single_fclayer_modelwrapper(W, pe, simd, wdt, idt, odt, T, tdt)
    for node in model.graph.node:
        # lookup op_type in registry of CustomOps
        inst = getCustomOp(node)
        inst.set_nodeattr("mem_mode", mem_mode)

    # prepare input data
    input_dict = prepare_inputs(x, idt, wdt)
    if wdt == DataType.BIPOLAR and idt == DataType.BIPOLAR:
        # convert inputs to binary and use xnorpopcountmatmul
        y = xp.xnorpopcountmatmul((x + 1) / 2, (W + 1) / 2)
    else:
        y = np.matmul(x, W)
    if T is not None:
        y = multithreshold(y, T)
        if act == DataType.BIPOLAR:
            # binary to bipolar
            y = 2 * y - 1
        else:
            # signed offset
            y += act.min()
    oshape = model.get_tensor_shape("outp")
    y_expected = y.reshape(oshape)
    # TODO split up into several dependent tests -- need to check how this
    # works for parametrized tests...
    model = model.transform(SetExecMode("rtlsim"))
    model = model.transform(GiveUniqueNodeNames())
    model = model.transform(PrepareIP("xc7z020clg400-1", 5))
    model = model.transform(HLSSynthIP())
    model = model.transform(PrepareRTLSim())
    y_produced = oxe.execute_onnx(model, input_dict)["outp"]
    assert (y_produced.reshape(y_expected.shape) == y_expected).all(), "rtlsim failed"

    hls_synt_res_est = model.analysis(hls_synth_res_estimation)
    assert "StreamingFCLayer_Batch_0" in hls_synt_res_est

    node = model.get_nodes_by_op_type("StreamingFCLayer_Batch")[0]
    inst = getCustomOp(node)
    cycles_rtlsim = inst.get_nodeattr("cycles_rtlsim")
    exp_cycles_dict = model.analysis(exp_cycles_per_layer)
    exp_cycles = exp_cycles_dict[node.name]
    assert np.isclose(exp_cycles, cycles_rtlsim, atol=15)
    assert exp_cycles != 0

In [7]:
def test_fpgadataflow_fclayer_synth(mem_mode, idt, wdt, act, nf, sf, mw, mh):
    if nf == -1:
        nf = mh
    if sf == -1:
        sf = mw
    pe = mh // nf
    simd = mw // sf
    assert mh % pe == 0
    assert mw % sf == 0
    # generate weights
    W = gen_finn_dt_tensor(wdt, (mw, mh))
    # generate input data
    x = gen_finn_dt_tensor(idt, (1, mw))
    if act is None:
        # no activation, produce accumulators
        T = None
        tdt = None
        if wdt == DataType.BIPOLAR and idt == DataType.BIPOLAR:
            odt = DataType.UINT32
        else:
            odt = DataType.INT32
    else:
        odt = act
        (min, max) = calculate_signed_dot_prod_range(idt, wdt, mw)
        n_steps = act.get_num_possible_values() - 1
        T = np.random.randint(min, max - 1, (mh, n_steps)).astype(np.float32)
        # provide non-decreasing thresholds
        T = np.sort(T, axis=1)
        # generate thresholds for activation
        if wdt == DataType.BIPOLAR and idt == DataType.BIPOLAR:
            tdt = DataType.UINT32
            # bias thresholds to be positive
            T = np.ceil((T + mw) / 2)
            assert (T >= 0).all()
        else:
            tdt = DataType.INT32
            
    model = make_single_fclayer_modelwrapper(W, pe, simd, wdt, idt, odt, T, tdt)
    for node in model.graph.node:
        # lookup op_type in registry of CustomOps
        inst = getCustomOp(node)
        inst.set_nodeattr("mem_mode", mem_mode)

    # prepare input data
    input_dict = prepare_inputs(x, idt, wdt)
    if wdt == DataType.BIPOLAR and idt == DataType.BIPOLAR:
        # convert inputs to binary and use xnorpopcountmatmul
        y = xp.xnorpopcountmatmul((x + 1) / 2, (W + 1) / 2)
    else:
        y = np.matmul(x, W)
    if T is not None:
        y = multithreshold(y, T)
        if act == DataType.BIPOLAR:
            # binary to bipolar
            y = 2 * y - 1
        else:
            # signed offset
            y += act.min()
    oshape = model.get_tensor_shape("outp")
    y_expected = y.reshape(oshape)
    # TODO split up into several dependent tests -- need to check how this
    # works for parametrized tests...
 
    model = model.transform(SetExecMode("rtlsim"))
    model = model.transform(GiveUniqueNodeNames())
    model = model.transform(PrepareIP("xc7z020clg400-1", 10))
    model = model.transform(HLSSynthIP())
    model = model.transform(PrepareRTLSim())
    y_produced = oxe.execute_onnx(model, input_dict)["outp"]
    assert (y_produced.reshape(y_expected.shape) == y_expected).all(), "rtlsim failed"
    

    node = model.get_nodes_by_op_type("StreamingFCLayer_Batch")[0]   
    inst = getCustomOp(node)
    inst.set_nodeattr("inFIFODepth", 8)
    inst.set_nodeattr("outFIFODepth", 8)
    #print(inst.get_nodeattr_types())
    #cycles_rtlsim = inst.get_nodeattr("cycles_rtlsim")
    

    model.save(build_dir+"/tfc_experiment.onnx")
    showInNetron(build_dir+"/tfc_experiment.onnx")
    
    #analysis
    exp_cycles_dict = model.analysis(exp_cycles_per_layer)
    exp_cycles = exp_cycles_dict[node.name]
    assert exp_cycles != 0
    print('Estimation of execution cycles is:', exp_cycles)
    
    res_util_dict = model.analysis(res_estimation)
    #res = res_util_dict[node.name]
    print('The Estimation of Resource Utilization is', res_util_dict)
    
       
    hls_synt_res_est = model.analysis(hls_synth_res_estimation)
    assert "StreamingFCLayer_Batch_0" in hls_synt_res_est
    print('Estimation of HLS Synth is', hls_synt_res_est)

In [255]:
test_fpgadataflow_fclayer_synth('const', DataType.INT4, DataType.INT4, DataType.INT4, 2, 1, 8, 8)

%Warning-STMTDLY: /home/chensun/workspace/bitfile/finn_dev_chensun/code_gen_ipgen_StreamingFCLayer_Batch_0_1g7zgblq/project_StreamingFCLayer_Batch_0/sol1/impl/verilog//StreamingFCLayer_Batch_0_Matrix_Vector_Activa.v:716: Unsupported: Ignoring delay on this delayed statement.
%Warning-STMTDLY: Use "/* verilator lint_off STMTDLY */" and lint_on around source to disable this message.
%Warning-STMTDLY: /home/chensun/workspace/bitfile/finn_dev_chensun/code_gen_ipgen_StreamingFCLayer_Batch_0_1g7zgblq/project_StreamingFCLayer_Batch_0/sol1/impl/verilog//StreamingFCLayer_Batch_0_Matrix_Vector_Activa.v:717: Unsupported: Ignoring delay on this delayed statement.
%Warning-STMTDLY: /home/chensun/workspace/bitfile/finn_dev_chensun/code_gen_ipgen_StreamingFCLayer_Batch_0_1g7zgblq/project_StreamingFCLayer_Batch_0/sol1/impl/verilog//StreamingFCLayer_Batch_0_Matrix_Vector_Activa.v:718: Unsupported: Ignoring delay on this delayed statement.
%Warning-STMTDLY: /home/chensun/workspace/bitfile/finn_dev_chens

make: Entering directory '/workspace/workspace/bitfile/finn_dev_chensun/pyverilator_StreamingFCLayer_Batch_0_34l8_27e'
g++  -I.  -MMD -I/usr/share/verilator/include -I/usr/share/verilator/include/vltstd -DVL_PRINTF=printf -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=1 -faligned-new -Wno-sign-compare -Wno-uninitialized -Wno-unused-but-set-variable -Wno-unused-parameter -Wno-unused-variable -Wno-shadow     -fPIC --std=c++11   -c -o pyverilator_wrapper.o /home/chensun/workspace/bitfile/finn_dev_chensun/pyverilator_StreamingFCLayer_Batch_0_34l8_27e/pyverilator_wrapper.cpp
g++  -I.  -MMD -I/usr/share/verilator/include -I/usr/share/verilator/include/vltstd -DVL_PRINTF=printf -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=1 -faligned-new -Wno-sign-compare -Wno-uninitialized -Wno-unused-but-set-variable -Wno-unused-parameter -Wno-unused-variable -Wno-shadow     -fPIC --std=c++11   -c -o verilated.o /usr/share/verilator/include/verilated.cpp
g++  -I.  -MMD -I/usr/share/verilator/include -I/usr/share/verilator/in

ar: creating VStreamingFCLayer_Batch_0_StreamingFCLayer_Batch_0__ALL.a


Stopping http://0.0.0.0:8081
Serving '/workspace/finn/tfc_experiment.onnx' at http://0.0.0.0:8081
Estimation of execution cycles is: 2
The Estimation of Resource Utilization is {'StreamingFCLayer_Batch_0': {'BRAM_18K': 0, 'BRAM_efficiency': 1, 'LUT': 3301, 'URAM': 0, 'URAM_efficiency': 1, 'DSP': 0}}
Estimation of HLS Synth is {'StreamingFCLayer_Batch_0': {'BRAM_18K': '0', 'FF': '658', 'LUT': '4215', 'DSP48E': '0', 'URAM': '0'}}


In [256]:
model = ModelWrapper(build_dir + "/tfc_experiment.onnx")
pynq_board = "Pynq-Z1"
fpga_part = pynq_part_map[pynq_board]
target_clk_ns = 10
model = model.transform(ZynqBuild(platform = pynq_board, period_ns = target_clk_ns))
model.save(build_dir+"/tfc_synthesis.onnx")

/workspace/finn-base/src/finn/transformation/infer_data_layouts.py:119: UserWarning: Assuming 2D input is NC
  warnings.warn("Assuming 2D input is NC")
/workspace/finn/src/finn/transformation/fpgadataflow/floorplan.py:107: UserWarning: 3 nodes have no entry in the provided floorplan, SLR was set to -1
  warnings.warn(
/workspace/finn/src/finn/transformation/fpgadataflow/prepare_ip.py:56: UserWarning: Using pre-existing code for StreamingDataflowPartition_1_StreamingFCLayer_Batch_0
  warnings.warn("Using pre-existing code for %s" % node.name)
/workspace/finn/src/finn/transformation/fpgadataflow/hlssynth_ip.py:70: UserWarning: Using pre-existing IP for StreamingDataflowPartition_1_StreamingFCLayer_Batch_0
  warnings.warn("Using pre-existing IP for %s" % node.name)


In [9]:
showInNetron(build_dir+"/tfc_synthesis.onnx")

Stopping http://0.0.0.0:8081
Serving '/workspace/finn/tfc_synthesis.onnx' at http://0.0.0.0:8081


In [257]:
model = ModelWrapper(build_dir + "/tfc_synthesis.onnx")
sdp_node_middle = getCustomOp(model.graph.node[1])
postsynth_layers = sdp_node_middle.get_nodeattr("model")

showInNetron(postsynth_layers)

Stopping http://0.0.0.0:8081
Serving '/home/chensun/workspace/bitfile/finn_dev_chensun/dataflow_partition2_y3chvub6/df_model.onnx' at http://0.0.0.0:8081


In [11]:
#model = ModelWrapper(postsynth_layers)
model.model.metadata_props

[key: "floorplan_json"
value: "/home/chensun/workspace/bitfile/finn_dev_chensun/vitis_floorplan_n6mu3yh3/floorplan.json"
, key: "vivado_pynq_proj"
value: "/home/chensun/workspace/bitfile/finn_dev_chensun/vivado_zynq_proj_723rk8sk"
, key: "bitfile"
value: "/home/chensun/workspace/bitfile/finn_dev_chensun/vivado_zynq_proj_723rk8sk/resizer.bit"
, key: "hw_handoff"
value: "/home/chensun/workspace/bitfile/finn_dev_chensun/vivado_zynq_proj_723rk8sk/resizer.hwh"
, key: "vivado_synth_rpt"
value: "/home/chensun/workspace/bitfile/finn_dev_chensun/vivado_zynq_proj_723rk8sk/synth_report.xml"
, key: "platform"
value: "zynq-iodma"
, key: "pynq_driver_dir"
value: "/home/chensun/workspace/bitfile/finn_dev_chensun/pynq_driver_3244cd2f"
]

In [239]:
import os

# set up the following values according to your own environment
# FINN will use ssh to deploy and run the generated accelerator
ip = os.getenv("PYNQ_IP", "192.168.31.175")
username = os.getenv("PYNQ_USERNAME", "xilinx")
password = os.getenv("PYNQ_PASSWORD", "xilinx")
port = os.getenv("PYNQ_PORT", 22)
target_dir = os.getenv("PYNQ_TARGET_DIR", "/home/xilinx/finn_tfc_end2end_example")
# set up ssh options to only allow publickey authentication
options = "-o PreferredAuthentications=publickey -o PasswordAuthentication=no"

# test access to PYNQ board
! ssh {options} {username}@{ip} -p {port} cat /var/run/motd.dynamic

Welcome to PYNQ Linux, based on Ubuntu 18.04 (GNU/Linux 4.19.0-xilinx-v2019.1 armv7l)

 * Super-optimized for small spaces - read how we shrank the memory
   footprint of MicroK8s to make it the smallest full K8s around.

   https://ubuntu.com/blog/microk8s-memory-optimisation


In [258]:
from finn.transformation.fpgadataflow.make_deployment import DeployToPYNQ

model = model.transform(DeployToPYNQ(ip, port, username, password, target_dir))
model.save(build_dir + "/tfc_experiment_deploy_index18.onnx")

In [174]:
target_dir_pynq = target_dir + "/" + model.get_metadata_prop("pynq_deployment_dir").split("/")[-1]
target_dir_pynq

'/home/xilinx/finn_dev_chensun/pynq_deployment_czstevg3'

In [114]:
model = ModelWrapper(build_dir + "/tfc_synthesis.onnx")
synth_res = model.analysis(post_synth_res)
print("Post Synthesis Resource Utilization is",synth_res)

Post Synthesis Resource Utilization is {'(top)': {'LUT': 6742, 'SRL': 180, 'FF': 8287, 'BRAM_36K': 1, 'BRAM_18K': 1, 'DSP48': 0}, 'StreamingDataflowPartition_0_IODMA_0': {'LUT': 817, 'SRL': 34, 'FF': 1603, 'BRAM_36K': 1, 'BRAM_18K': 0, 'DSP48': 0}, 'StreamingDataflowPartition_1_StreamingFIFO_0': {'LUT': 19, 'SRL': 4, 'FF': 12, 'BRAM_36K': 0, 'BRAM_18K': 0, 'DSP48': 0}, 'StreamingDataflowPartition_1_StreamingFCLayer_Batch_0': {'LUT': 1473, 'SRL': 0, 'FF': 405, 'BRAM_36K': 0, 'BRAM_18K': 0, 'DSP48': 0}, 'StreamingDataflowPartition_1_StreamingFIFO_1': {'LUT': 35, 'SRL': 8, 'FF': 24, 'BRAM_36K': 0, 'BRAM_18K': 0, 'DSP48': 0}, 'StreamingDataflowPartition_2_IODMA_0': {'LUT': 878, 'SRL': 40, 'FF': 1765, 'BRAM_36K': 0, 'BRAM_18K': 1, 'DSP48': 0}}


In [260]:
from finn.core.throughput_test import throughput_test_remote

res = throughput_test_remote(model, 1)
print("Network metrics:")
for key in res:
    print(str(key) + ": " + str(res[key]))

[sudo] password for xilinx: 

Network metrics:
runtime[ms]: 0.3085136413574219
throughput[images/s]: 3241.3477588871715
DRAM_in_bandwidth[Mb/s]: 0.012965391035548685
DRAM_out_bandwidth[Mb/s]: 0.012965391035548685
fclk[mhz]: 100.0
batch_size: 1
fold_input[ms]: 0.08559226989746094
pack_input[ms]: 9.696245193481445
copy_input_data_to_device[ms]: 1.6405582427978516
copy_output_data_from_device[ms]: 0.5311965942382812
unpack_output[ms]: 2.686023712158203
unfold_output[ms]: 0.061511993408203125


In [83]:
II = 32
# frequency in MHz
f_MHz = 100
# expected throughput in MFPS
expected_throughput = f_MHz / II
# measured throughput (FPS) from throughput test, converted to MFPS
measured_throughput = res["throughput[images/s]"] * 0.000001
# peformance
print("We reach approximately " + str(round((measured_throughput / expected_throughput)*100)) + "% of the ideal performance.")

We reach approximately 84% of the ideal performance.
